In [2]:
#importing numpy and panda libraries
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

#reading the data from adult dataset url
adultdf=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data')
adultdf.head(3) # displays first 3 data


,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K


In [3]:
#Renaming the columns as per the description from this file: 
#https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
adultdf.rename(columns={'39':'age',' State-gov': 'workclass',' 77516':'fnlwgt',' Bachelors':'education',' 13':'education-num',
' Never-married':'marital-status',         
' Adm-clerical':'occupation',      
' Not-in-family':'relationship',    
' White':'race',            
' Male':'sex',              
' 2174':'capital-gain',            
' 0':'capital-loss',               
' 40':'hours-per-week',               
' United-States':'native-country',    
' <=50K':'label'},inplace=True)
adultdf.head(3)  # displays first 3 data


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K


In [4]:
#Create a sql db from adult dataset and name it sqladb
#importing panda sql libraries
from pandasql import sqldf
from pandas import read_sql_query
import sqlite3 as db
from sqlite3 import Error

#define the global function, as managing local and global environment might be confusing at times
pysqldf = lambda q: sqldf(q, locals())

conn = db.connect('sqladb.db') # creating connection to database sqladb
c = conn.cursor() # Cursor to execute the queries

In [6]:
#Creating AdultTable
q="""CREATE TABLE adultTable (
            age   int,
            workclass   varchar(40),
            fnlwgt   int,
            education   varchar(40),
            education_num   int,
            marital_status   varchar(40),
            occupation   varchar(40),
            relationship   varchar(40),
            race   varchar(20),
            sex   varchar(10),
            capital_gain   int,
            capital_loss   int,
            hours_per_week   int,
            native_country   varchar(50),
            label    varchar(10))"""

c.execute(q)
print("Table created successfully")

Table created successfully


In [8]:
# inserting dataframe data into adultTable in sqladb
for index,row in adultdf.iterrows():    
    c.executemany('INSERT INTO adultTable VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)',[tuple(row.values)])
    
conn.commit()
print('Data inserted Successfully')

Data inserted Successfully


In [9]:
#1. Select 10 records from the adult sqladb 
q="SELECT * FROM adultTable LIMIT 10;"
df = read_sql_query(q, conn)  # Reading the sql query and converting to dataframe
df

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,label
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
5,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
6,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
7,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
8,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K
9,37,Private,280464,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,80,United-States,>50K


In [87]:
#2. Show me the average hours per week of all men who are working in private sector 
q="SELECT AVG(hours_per_week) average_hours_per_week FROM adultTable WHERE workclass = ' Private' AND sex=' Male'"
resultdf = read_sql_query(q, conn) # Reading the sql query and converting to dataframe
resultdf

,average_hours_per_week
0,42.221226


In [88]:
#3. Show me the frequency table for education, occupation and relationship, separately 
# education
q="SELECT education , COUNT(education) frequency FROM adultTable GROUP BY education "
edf = read_sql_query(q, conn) # Reading the sql query and converting to dataframe
edf


,education,frequency
0,10th,933
1,11th,1175
2,12th,433
3,1st-4th,168
4,5th-6th,333
5,7th-8th,646
6,9th,514
7,Assoc-acdm,1067
8,Assoc-voc,1382
9,Bachelors,5354


In [89]:
# occupation
q="SELECT occupation, COUNT(occupation) frequency FROM adultTable GROUP BY occupation "
odf = read_sql_query(q, conn) # Reading the sql query and converting to dataframe
odf


,occupation,frequency
0,?,1843
1,Adm-clerical,3769
2,Armed-Forces,9
3,Craft-repair,4099
4,Exec-managerial,4066
5,Farming-fishing,994
6,Handlers-cleaners,1370
7,Machine-op-inspct,2002
8,Other-service,3295
9,Priv-house-serv,149


In [90]:
# relationship
q="SELECT relationship, COUNT(relationship) frequency FROM adultTable GROUP BY relationship "
rdf = read_sql_query(q, conn) # Reading the sql query and converting to dataframe
rdf


,relationship,frequency
0,Husband,13193
1,Not-in-family,8304
2,Other-relative,981
3,Own-child,5068
4,Unmarried,3446
5,Wife,1568


In [91]:
#4. Are there any people who are married, working in private sector and having a masters degree 
q="""SELECT CASE WHEN countVal>0 THEN 'Yes' ELSE 'No' END _exist FROM 
            (SELECT COUNT(marital_status) countVal FROM adultTable 
            WHERE marital_status IN (' Married-civ-spouse', ' Married-spouse-absent' , ' Married-AF-spouse') 
            AND workclass=' Private' AND education=' Masters' )"""
resultdf = read_sql_query(q, conn) # Reading the sql query and converting to dataframe
resultdf

,_exist
0,Yes


In [92]:
#5. What is the average, minimum and maximum age group for people working in different sectors 
q=" SELECT workclass as sector, AVG(age) averageAge, MIN(age) minAge, MAX(age) maxAge  FROM adultTable GROUP BY workclass "
resultdf = read_sql_query(q, conn) # Reading the sql query and converting to dataframe
resultdf


,sector,averageAge,minAge,maxAge
0,?,40.960240,17,90
1,Federal-gov,42.590625,17,90
2,Local-gov,41.751075,17,90
3,Never-worked,20.571429,17,30
4,Private,36.797585,17,90
5,Self-emp-inc,46.017025,17,84
6,Self-emp-not-inc,44.969697,17,90
7,State-gov,39.436392,17,81
8,Without-pay,47.785714,19,72


In [93]:
#6. Calculate age distribution by country 
q=" SELECT age, native_country FROM adultTable GROUP BY native_country "
resultdf = read_sql_query(q, conn) # Reading the sql query and converting to dataframe
resultdf

,age,native_country
0,81,?
1,48,Cambodia
2,32,Canada
3,23,China
4,49,Columbia
5,48,Cuba
6,45,Dominican-Republic
7,27,Ecuador
8,39,El-Salvador
9,48,England


In [11]:
#7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and 'capital-loss'

#Net-Capital-Gain= (capital-gain)-(capital-loss)
c.execute(" ALTER TABLE adultTable ADD net_capital_gain int;")
print("Added Net Capital Gain")


Added Net Capital Gain


In [12]:
#checking the added column
q="PRAGMA table_info([adultTable]);"
resultdf = read_sql_query(q, conn) # Reading the sql query and converting to dataframe
resultdf

,cid,name,type,notnull,dflt_value,pk
0,0,age,int,0,None,0
1,1,workclass,varchar(40),0,None,0
2,2,fnlwgt,int,0,None,0
3,3,education,varchar(40),0,None,0
4,4,education_num,int,0,None,0
5,5,marital_status,varchar(40),0,None,0
6,6,occupation,varchar(40),0,None,0
7,7,relationship,varchar(40),0,None,0
8,8,race,varchar(20),0,None,0
9,9,sex,varchar(10),0,None,0


In [13]:
#inserting  new Net_Capital_Gain column in dataframe
adultdf.insert(15, 'net_capital_gain', adultdf['capital-gain']-adultdf['capital-loss'])
adultdf.head()  # displays first 5 data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label,net_capital_gain
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,0
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,0
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,0
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,0
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K,0


In [14]:
#inserting Net_Capital_Gain column values in adultTable in sqladb
for index,row in adultdf.iterrows(): 
    net_capital_gain=row['capital-gain'] -row['capital-loss']
    c.execute('INSERT INTO adultTable (Net_Capital_Gain) VALUES(?)',[net_capital_gain])
conn.commit()

print("Net_Capital_Gain column values inserted Successfully")


Net_Capital_Gain column values inserted Successfully


In [15]:
conn.close(); # closing the connection